In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from joblib import dump

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
set_random_seed(12)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

# Preprocessing

In [2]:
data = pd.read_csv('./datasets/fundamental_data.csv')

In [3]:
y_data = data['growth']
data = data.drop(columns='growth')

X_train, X_test, Y_train, Y_test = train_test_split(data, y_data, test_size=0.2, random_state=12)

X_train.shape

(23857, 98)

In [4]:
# Normalise data
normaliser = preprocessing.MinMaxScaler()
X_train = normaliser.fit_transform(X_train)
X_test = normaliser.transform(X_test)

# Y normaliser
y_normaliser = preprocessing.MinMaxScaler()
Y_train = y_normaliser.fit_transform(Y_train.to_numpy().reshape(-1, 1))
Y_test = y_normaliser.transform(Y_test.to_numpy().reshape(-1, 1))

In [5]:
# Save scalers for future use
dump(normaliser, './normalisers/fundamental_x_normaliser.joblib')
dump(y_normaliser, './normalisers/fundamental_y_normaliser.joblib')

['./normalisers/fundamental_y_normaliser.joblib']

# Model selection

##### Model constructor

In [6]:
def build_dense(x, y, batch_size=512, epochs=24, **params):
    
    # List of parameters
    if 'density' not in params: params['density'] = x.shape[1]
    if 'activation' not in params: params['activation'] = 'relu'
    if 'optimizer' not in params: params['optimizer'] = 'adam'
    if 'shuffle' not in params: params['shuffle'] = False
    
    # Model definition
    model = Sequential()
    
    model.add(Dense(params['density'], input_shape= (98,), activation=params['activation']))
    
    density = params['density']//2
    while density >= 12:
        if 'dropout' in params:
            model.add(Dropout(params['dropout']))
        model.add(Dense(density, activation=params['activation']))
        density //= 2
            
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mse', optimizer=params['optimizer'])
    
    if 'callbacks' in params:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'],
                            callbacks=params['callbacks'])
    else:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'])
    
    return [model, callback, params]

##### Evolutive algorith to search for the most optimal model

In [7]:
def set_genes(x, y, population_size, population=[]):
    if population_size-len(population) < 0:
        print('population size must be greater or equal than actual population')
        return
    
    if len(population) > 0:
        if len(population) < 3:
            print('population should be empty or 3 at least')
            return
        
        population[2] = breed_genes(x, y, population[0][2].copy())
        population[1] = breed_genes(x, y, combine=(population[0][2], population[1][2]))
        population = population[:3]
    
    for _ in range(population_size-len(population)):
        subject = breed_genes(x, y)
        population.append(subject)
        
    return population

def breed_genes(x, y, genes={}, combine=None):
    genes['x'] = x
    genes['y'] = y
    
    if type(combine) is list or type(combine) is tuple:
        genes['density'] = combine[np.random.randint(0,2)]['density']
        genes['activation'] = combine[np.random.randint(0,2)]['activation']
        genes['optimizer'] = combine[np.random.randint(0,2)]['optimizer']
        genes['shuffle'] = combine[np.random.randint(0,2)]['shuffle']
        
        genes['dropout'] = combine[np.random.randint(0,2)].get('dropout')
        if genes['dropout'] is None: del genes['dropout']
            
    else:
        if np.random.randint(0,3) == 1:
            genes['density'] = int((np.random.randint(x.shape[1]//2,x.shape[1]*2.66)//2)*2)

        if np.random.randint(0,3) == 1:
            activation = [
                'relu',
                'sigmoid',
                'softplus',
                'softsign',
                'tanh',
                'selu',
                'elu',
                'exponential'
            ]
            genes['activation'] = activation[np.random.randint(0,8)]

        if np.random.randint(0,3) == 1:
            optimizer = [
                'sgd',
                'rmsprop',
                'adam',
                'adadelta',
                'adagrad',
                'adamax',
                'nadam'
            ]
            genes['optimizer'] = optimizer[np.random.randint(0,7)]

        if np.random.randint(0,3) == 1:
            genes['shuffle'] = True

        if np.random.randint(0,3) == 1:
            genes['dropout'] = np.random.randint(1,4)/10
            
    new_model = build_dense(**genes)
        
    return new_model

In [8]:
def breed_population(x, y, generations, population_size, population=[]):
    
    if generations > 1 and population_size < 3:
        print('population size should be of a minimum of 3 for more than one generation')
        return
    
    for g in range(generations):
        print(f'\nGENERATION {g}\n')
        population = set_genes(x, y, population_size, population)
        population = sorted(population, key=lambda x: x[1].history['val_loss'][-1]+x[1].history['loss'][-1])
        
    return population

In [9]:
best_models = breed_population(X_train, Y_train, generations=20, population_size=10)


GENERATION 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 1s 32us/step - loss: 0.0037 - val_loss: 3.4480e-05
Epoch 2/24
21471/21471 [==============================] - 0s 5us/step - loss: 9.0661e-05 - val_loss: 3.4474e-05
Epoch 3/24
21471/21471 [==============================] - 0s 5us/step - loss: 9.0677e-05 - val_loss: 3.4482e-05
Epoch 4/24
21471/21471 [==============================] - 0s 5us/step - loss: 9.0647e-05 - val_loss: 3.4474e-05
Epoch 5/24
21471/21471 [==============================] - 0s 5us/step - loss: 9.0671e-05 - val_loss: 3.4481e-05
Epoch 6/24
21471/21471 [==============================] - 0s 5us/step - loss: 9.0664e-05 - val_loss: 3.4495e-05
Epoch 7/24
21471/21471 [==============================] - 0s 5us/step - loss: 9.0668e-05 - val_loss: 3.4507e-05
Epoch 8/24
21471/21471 [===========

21471/21471 [==============================] - 0s 5us/step - loss: 1.0348e-04 - val_loss: 4.4441e-05
Epoch 20/24
21471/21471 [==============================] - 0s 5us/step - loss: 1.0345e-04 - val_loss: 4.4341e-05
Epoch 21/24
21471/21471 [==============================] - 0s 4us/step - loss: 1.0349e-04 - val_loss: 4.4329e-05
Epoch 22/24
21471/21471 [==============================] - 0s 5us/step - loss: 1.0339e-04 - val_loss: 4.4380e-05
Epoch 23/24
21471/21471 [==============================] - 0s 5us/step - loss: 1.0344e-04 - val_loss: 4.4468e-05
Epoch 24/24
21471/21471 [==============================] - 0s 5us/step - loss: 1.0341e-04 - val_loss: 4.4298e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 0s 13us/step - loss: 0.0131 - val_loss: 5.7720e-05
Epoch 2/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0026 - val_loss: 0.0153
Epoch 3/24
21471/21471 [==============================] - 0s 5us/step - 

Epoch 23/24
21471/21471 [==============================] - 0s 6us/step - loss: 7.8548e-04 - val_loss: 9.8642e-04
Epoch 24/24
21471/21471 [==============================] - 0s 6us/step - loss: 7.0891e-04 - val_loss: 5.3329e-04
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 0s 19us/step - loss: 0.0728 - val_loss: 2.0127e-04
Epoch 2/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0144 - val_loss: 2.9158e-04
Epoch 3/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0061 - val_loss: 1.9898e-04
Epoch 4/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0024 - val_loss: 4.3866e-04
Epoch 5/24
21471/21471 [==============================] - 0s 6us/step - loss: 9.2294e-04 - val_loss: 2.2185e-04
Epoch 6/24
21471/21471 [==============================] - 0s 6us/step - loss: 4.8196e-04 - val_loss: 1.6586e-04
Epoch 7/24
21471/21471 [==============================] - 0s 6us/ste

21471/21471 [==============================] - 0s 6us/step - loss: nan - val_loss: nan
Epoch 5/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 6/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 7/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 8/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 9/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 10/24
21471/21471 [==============================] - 0s 6us/step - loss: nan - val_loss: nan
Epoch 11/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 12/24
21471/21471 [==============================] - 0s 7us/step - loss: nan - val_loss: nan
Epoch 13/24
21471/21471 [==============================] - 0s 6us/step - loss: nan - val_loss: nan
Epoch 14/24
21471/21471 [==

21471/21471 [==============================] - 0s 7us/step - loss: 1.1693e-04 - val_loss: 1.9917e-04
Epoch 7/24
21471/21471 [==============================] - 0s 7us/step - loss: 2.9863e-04 - val_loss: 3.6013e-05
Epoch 8/24
21471/21471 [==============================] - 0s 7us/step - loss: 9.6183e-05 - val_loss: 3.5303e-05
Epoch 9/24
21471/21471 [==============================] - 0s 8us/step - loss: 1.8232e-04 - val_loss: 2.6862e-04
Epoch 10/24
21471/21471 [==============================] - 0s 8us/step - loss: 1.2036e-04 - val_loss: 4.6833e-05
Epoch 11/24
21471/21471 [==============================] - 0s 8us/step - loss: 1.0142e-04 - val_loss: 3.6935e-05
Epoch 12/24
21471/21471 [==============================] - 0s 7us/step - loss: 9.5842e-05 - val_loss: 4.1097e-05
Epoch 13/24
21471/21471 [==============================] - 0s 7us/step - loss: 2.2973e-04 - val_loss: 3.7480e-05
Epoch 14/24
21471/21471 [==============================] - 0s 7us/step - loss: 9.0489e-05 - val_loss: 3.3826e-0

...

21471/21471 [==============================] - 0s 18us/step - loss: 0.0019 - val_loss: 6.0365e-05
Epoch 24/24
21471/21471 [==============================] - 0s 18us/step - loss: 0.0018 - val_loss: 7.0959e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 6s 303us/step - loss: 0.1404 - val_loss: 0.0025
Epoch 2/24
21471/21471 [==============================] - 0s 19us/step - loss: 0.0337 - val_loss: 0.0050
Epoch 3/24
21471/21471 [==============================] - 0s 19us/step - loss: 0.0229 - val_loss: 0.0025
Epoch 4/24
21471/21471 [==============================] - 0s 18us/step - loss: 0.0174 - val_loss: 0.0014
Epoch 5/24
21471/21471 [==============================] - 0s 20us/step - loss: 0.0141 - val_loss: 0.0012
Epoch 6/24
21471/21471 [==============================] - 0s 19us/step - loss: 0.0120 - val_loss: 7.8091e-04
Epoch 7/24
21471/21471 [==============================] - 0s 18us/step - loss: 0.0102 - val_loss: 5.2599e-04

21471/21471 [==============================] - 0s 19us/step - loss: 0.0018 - val_loss: 8.0978e-05

GENERATION 19

Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 7s 315us/step - loss: 6.2600e-04 - val_loss: 3.6474e-05
Epoch 2/24
21471/21471 [==============================] - 0s 18us/step - loss: 9.1643e-05 - val_loss: 3.1586e-05
Epoch 3/24
21471/21471 [==============================] - 0s 18us/step - loss: 8.7951e-05 - val_loss: 3.2264e-05
Epoch 4/24
21471/21471 [==============================] - 0s 18us/step - loss: 8.8125e-05 - val_loss: 3.1597e-05
Epoch 5/24
21471/21471 [==============================] - 0s 18us/step - loss: 8.8039e-05 - val_loss: 3.1486e-05
Epoch 6/24
21471/21471 [==============================] - 0s 18us/step - loss: 8.7995e-05 - val_loss: 3.1470e-05
Epoch 7/24
21471/21471 [==============================] - 0s 18us/step - loss: 8.7955e-05 - val_loss: 3.2245e-05
Epoch 8/24
21471/21471 [=====================

Epoch 23/24
21471/21471 [==============================] - 0s 20us/step - loss: 8.7895e-05 - val_loss: 3.1463e-05
Epoch 24/24
21471/21471 [==============================] - 0s 19us/step - loss: 8.7844e-05 - val_loss: 3.1485e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 7s 335us/step - loss: 0.0016 - val_loss: 1.0700e-04
Epoch 2/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.6144e-04 - val_loss: 3.6435e-04
Epoch 3/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.7168e-04 - val_loss: 4.2993e-04
Epoch 4/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.4858e-04 - val_loss: 4.3525e-04
Epoch 5/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.3694e-04 - val_loss: 3.3619e-04
Epoch 6/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.1490e-04 - val_loss: 3.1607e-04
Epoch 7/24
21471/21471 [========================

Epoch 23/24
21471/21471 [==============================] - 0s 19us/step - loss: 9.5596e-04 - val_loss: 7.3371e-05
Epoch 24/24
21471/21471 [==============================] - 0s 20us/step - loss: 8.7418e-04 - val_loss: 8.0916e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 8s 351us/step - loss: 0.0179 - val_loss: 1.7698e-04
Epoch 2/24
21471/21471 [==============================] - 0s 20us/step - loss: 0.0063 - val_loss: 1.3025e-04
Epoch 3/24
21471/21471 [==============================] - 0s 19us/step - loss: 0.0037 - val_loss: 9.4171e-05
Epoch 4/24
21471/21471 [==============================] - 0s 21us/step - loss: 0.0022 - val_loss: 6.8667e-05
Epoch 5/24
21471/21471 [==============================] - 0s 21us/step - loss: 0.0015 - val_loss: 4.8180e-05
Epoch 6/24
21471/21471 [==============================] - 0s 20us/step - loss: 0.0011 - val_loss: 4.9264e-05
Epoch 7/24
21471/21471 [==============================] - 0s 22us/st

21471/21471 [==============================] - 0s 21us/step - loss: 2.0648e-04 - val_loss: 5.7903e-05
Epoch 23/24
21471/21471 [==============================] - 0s 20us/step - loss: 1.2848e-04 - val_loss: 3.9290e-05
Epoch 24/24
21471/21471 [==============================] - 0s 22us/step - loss: 1.4616e-04 - val_loss: 3.5053e-05


In [10]:
best_models[0][1].history['val_loss']

[0.00034386917341957987,
 3.783172801830681e-05,
 3.1464102870774164e-05,
 3.1470912837761796e-05,
 3.147362602970096e-05,
 3.145755387235134e-05,
 3.146474106008072e-05,
 3.145733422366317e-05,
 3.145742804341629e-05,
 3.153769932384047e-05,
 3.1465728658497666e-05,
 3.1459942150548324e-05,
 3.145837165293651e-05,
 3.145886018805368e-05,
 3.151930285024313e-05,
 3.146486582476212e-05,
 3.1501187267185085e-05,
 3.154301745574086e-05,
 3.1505211624956766e-05,
 3.1472171146171304e-05,
 3.145982097276765e-05,
 3.1525540156521935e-05,
 3.145735740319913e-05,
 3.1468058131344e-05]

In [11]:
print('optimal parameters')
print('_______________________')
for key,val in best_models[0][2].items():
    if key != 'x' and key != 'y': print(f'{key}: {val}')

optimal parameters
_______________________
activation: sigmoid
optimizer: adamax
shuffle: True
density: 148


In [12]:
best_models[0][0].summary()

Model: "sequential_84"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_432 (Dense)            (None, 148)               14652     
_________________________________________________________________
dense_433 (Dense)            (None, 74)                11026     
_________________________________________________________________
dense_434 (Dense)            (None, 37)                2775      
_________________________________________________________________
dense_435 (Dense)            (None, 18)                684       
_________________________________________________________________
dense_436 (Dense)            (None, 1)                 19        
Total params: 29,156
Trainable params: 29,156
Non-trainable params: 0
_________________________________________________________________


In [13]:
checkpoint = ModelCheckpoint(
    filepath='./checkpoints/fundamental.{epoch}-{val_loss}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [14]:
params = best_models[0][2]
params['x'] = X_train
params['y'] = Y_train
params['epochs'] = 2000
params['callbacks'] = [checkpoint]

In [15]:
final_model = build_dense(**params)

Train on 21471 samples, validate on 2386 samples
Epoch 1/2000
21471/21471 [==============================] - 8s 350us/step - loss: 0.0162 - val_loss: 6.4895e-04
Epoch 2/2000
21471/21471 [==============================] - 0s 19us/step - loss: 1.6463e-04 - val_loss: 3.9912e-05
Epoch 3/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.9299e-05 - val_loss: 3.1514e-05
Epoch 4/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.7873e-05 - val_loss: 3.1490e-05
Epoch 5/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.7868e-05 - val_loss: 3.1477e-05
Epoch 6/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.7884e-05 - val_loss: 3.1485e-05
Epoch 7/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.7869e-05 - val_loss: 3.1480e-05
Epoch 8/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.7880e-05 - val_loss: 3.1482e-05
Epoch 9/2000
21471/21471 [========

21471/21471 [==============================] - 0s 20us/step - loss: 8.8461e-05 - val_loss: 3.1929e-05
Epoch 72/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8149e-05 - val_loss: 3.1942e-05
Epoch 73/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8466e-05 - val_loss: 3.1553e-05
Epoch 74/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8431e-05 - val_loss: 3.1884e-05
Epoch 75/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8616e-05 - val_loss: 3.1538e-05
Epoch 76/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8079e-05 - val_loss: 3.1874e-05
Epoch 77/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8013e-05 - val_loss: 3.1455e-05
Epoch 78/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.7982e-05 - val_loss: 3.1941e-05
Epoch 79/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.80

21471/21471 [==============================] - 0s 19us/step - loss: 8.9289e-05 - val_loss: 3.5389e-05
Epoch 142/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.2328e-05 - val_loss: 3.2296e-05
Epoch 143/2000
21471/21471 [==============================] - 0s 19us/step - loss: 9.0166e-05 - val_loss: 3.2988e-05
Epoch 144/2000
21471/21471 [==============================] - 0s 19us/step - loss: 9.3072e-05 - val_loss: 3.1674e-05
Epoch 145/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.8902e-05 - val_loss: 3.1671e-05
Epoch 146/2000
21471/21471 [==============================] - 0s 19us/step - loss: 9.0067e-05 - val_loss: 3.2683e-05
Epoch 147/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.0131e-05 - val_loss: 3.7898e-05
Epoch 148/2000
21471/21471 [==============================] - 0s 19us/step - loss: 9.0721e-05 - val_loss: 5.0255e-05
Epoch 149/2000
21471/21471 [==============================] - 0s 18us/step - lo

21471/21471 [==============================] - 0s 19us/step - loss: 8.9623e-05 - val_loss: 3.1731e-05
Epoch 212/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.9123e-05 - val_loss: 3.3091e-05
Epoch 213/2000
21471/21471 [==============================] - 0s 18us/step - loss: 8.9385e-05 - val_loss: 3.1507e-05
Epoch 214/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.0083e-05 - val_loss: 3.3872e-05
Epoch 215/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.1573e-05 - val_loss: 3.4836e-05
Epoch 216/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.1015e-05 - val_loss: 3.2465e-05
Epoch 217/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.1934e-05 - val_loss: 3.1681e-05
Epoch 218/2000
21471/21471 [==============================] - 0s 18us/step - loss: 9.0991e-05 - val_loss: 3.2684e-05
Epoch 219/2000
21471/21471 [==============================] - 0s 18us/step - lo

21471/21471 [==============================] - 0s 20us/step - loss: 9.0323e-05 - val_loss: 3.6497e-05
Epoch 282/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.9514e-05 - val_loss: 3.1449e-05
Epoch 283/2000
21471/21471 [==============================] - 0s 20us/step - loss: 9.0759e-05 - val_loss: 3.4217e-05
Epoch 284/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.9344e-05 - val_loss: 3.1523e-05
Epoch 285/2000
21471/21471 [==============================] - 0s 20us/step - loss: 9.2753e-05 - val_loss: 3.2261e-05
Epoch 286/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8839e-05 - val_loss: 3.3096e-05
Epoch 287/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8808e-05 - val_loss: 3.1636e-05
Epoch 288/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8376e-05 - val_loss: 3.1470e-05
Epoch 289/2000
21471/21471 [==============================] - 0s 20us/step - lo

...

21471/21471 [==============================] - 0s 20us/step - loss: 8.9689e-05 - val_loss: 3.2490e-05
Epoch 1672/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8065e-05 - val_loss: 3.1521e-05
Epoch 1673/2000
21471/21471 [==============================] - 0s 22us/step - loss: 8.8090e-05 - val_loss: 3.1465e-05
Epoch 1674/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8113e-05 - val_loss: 3.1811e-05
Epoch 1675/2000
21471/21471 [==============================] - 0s 20us/step - loss: 9.0492e-05 - val_loss: 3.1576e-05
Epoch 1676/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.9200e-05 - val_loss: 3.1869e-05
Epoch 1677/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8494e-05 - val_loss: 3.2082e-05
Epoch 1678/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8292e-05 - val_loss: 3.1820e-05
Epoch 1679/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 21us/step - loss: 8.8669e-05 - val_loss: 3.2064e-05
Epoch 1741/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8192e-05 - val_loss: 3.1517e-05
Epoch 1742/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8466e-05 - val_loss: 3.2146e-05
Epoch 1743/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8890e-05 - val_loss: 3.1483e-05
Epoch 1744/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8428e-05 - val_loss: 3.2473e-05
Epoch 1745/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.9412e-05 - val_loss: 3.6998e-05
Epoch 1746/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.9158e-05 - val_loss: 3.1602e-05
Epoch 1747/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.7908e-05 - val_loss: 3.2663e-05
Epoch 1748/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 20us/step - loss: 8.8339e-05 - val_loss: 3.1909e-05
Epoch 1810/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.9064e-05 - val_loss: 3.1463e-05
Epoch 1811/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8297e-05 - val_loss: 3.1457e-05
Epoch 1812/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8578e-05 - val_loss: 3.1647e-05
Epoch 1813/2000
21471/21471 [==============================] - 0s 19us/step - loss: 9.0199e-05 - val_loss: 3.1716e-05
Epoch 1814/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8897e-05 - val_loss: 3.1766e-05
Epoch 1815/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8219e-05 - val_loss: 3.1523e-05
Epoch 1816/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8601e-05 - val_loss: 3.1474e-05
Epoch 1817/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 19us/step - loss: 8.8823e-05 - val_loss: 3.1616e-05
Epoch 1879/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.8641e-05 - val_loss: 3.1931e-05
Epoch 1880/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8821e-05 - val_loss: 3.1970e-05
Epoch 1881/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8631e-05 - val_loss: 3.1465e-05
Epoch 1882/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.8539e-05 - val_loss: 3.3140e-05
Epoch 1883/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8713e-05 - val_loss: 3.1455e-05
Epoch 1884/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8833e-05 - val_loss: 3.3294e-05
Epoch 1885/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8736e-05 - val_loss: 3.3108e-05
Epoch 1886/2000
21471/21471 [==============================] - 0s 19us/s

21471/21471 [==============================] - 0s 20us/step - loss: 8.8498e-05 - val_loss: 3.5396e-05
Epoch 1948/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.8753e-05 - val_loss: 3.1458e-05
Epoch 1949/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8210e-05 - val_loss: 3.1506e-05
Epoch 1950/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8567e-05 - val_loss: 3.2086e-05
Epoch 1951/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8573e-05 - val_loss: 3.1636e-05
Epoch 1952/2000
21471/21471 [==============================] - 0s 21us/step - loss: 8.8764e-05 - val_loss: 3.1975e-05
Epoch 1953/2000
21471/21471 [==============================] - 0s 19us/step - loss: 8.9258e-05 - val_loss: 3.6459e-05
Epoch 1954/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.9343e-05 - val_loss: 3.3197e-05
Epoch 1955/2000
21471/21471 [==============================] - 0s 20us/s

In [16]:
final_model[2]

{'activation': 'sigmoid',
 'optimizer': 'adamax',
 'shuffle': True,
 'density': 148,
 'callbacks': [<keras.callbacks.callbacks.ModelCheckpoint at 0x2b0d9fd0c88>]}

In [17]:
final_model[0].summary()

Model: "sequential_182"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_923 (Dense)            (None, 148)               14652     
_________________________________________________________________
dense_924 (Dense)            (None, 74)                11026     
_________________________________________________________________
dense_925 (Dense)            (None, 37)                2775      
_________________________________________________________________
dense_926 (Dense)            (None, 18)                684       
_________________________________________________________________
dense_927 (Dense)            (None, 1)                 19        
Total params: 29,156
Trainable params: 29,156
Non-trainable params: 0
_________________________________________________________________


In [32]:
final_model[0].load_weights('./checkpoints/fundamental.(best).hdf5')
y_predicted = final_model[0].predict(X_test)

In [33]:
predicted_y_test = y_normaliser.inverse_transform(y_predicted)
true_y_test = y_normaliser.inverse_transform(Y_test)

In [34]:
loss = np.mean((true_y_test - predicted_y_test)**2)
medium_error = np.mean(abs(true_y_test - predicted_y_test))
print(f'MSE is {loss:.2f}\nMedium error is {medium_error:.2f}')

MSE is 87344.15
Medium error is 35.87


In [35]:
print(f'Trend accuracy is: {(sum(1 for r,p in zip(predicted_y_test,true_y_test) if (r > 0 and p > 0) or (r < 0 and p < 0))/len(Y_test))*100:.2f}%')

Trend accuracy is: 52.05%


In [36]:
print('Test growth trends:        Predicted growth trends:')
print(sum(1 for v in true_y_test if v > 0),'                     ',sum(1 for v in predicted_y_test if v > 0))
print('Test decreasing trends:    Predicted decreasing trends:')
print(sum(1 for v in true_y_test if v < 0),'                     ',sum(1 for v in predicted_y_test if v < 0))

Test growth trends:        Predicted growth trends:
3110                       5892
Test decreasing trends:    Predicted decreasing trends:
2855                       73


In [37]:
print(f'Test mean: {np.mean(true_y_test)}')
print(f'Predictions mean: {np.mean(predicted_y_test)}')

Test mean: 13.814097446110715
Predictions mean: 13.536017417907715


##### Results seem to be random around some tipical values

In [38]:
print('Test values:    Predicted values:')
for r,p in zip(true_y_test.flatten(),predicted_y_test.flatten()):
    print(f'    {r:.2f}    {p:>12.2f}')

Test values:    Predicted values:
    6.71           13.52
    15.98           14.10
    -24.16           14.20
    -3.26           14.31
    -37.74           14.32
    19.17           12.33
    -22.51           14.32
    3.10           14.39
    -32.61           14.39
    -2.50           14.45
    -35.70           14.40
    5.68           -4.87
    5.03           14.16
    42.44           14.20
    4.46            5.34
    -29.52           14.30
    7.42           14.35
    -4.15           -1.40
    -16.67           14.35
    37.96           14.33
    -21.83           14.27
    -5.49           14.28
    3.60           13.84
    17.70           14.18
    7.64           14.29
    19.82           14.37
    -11.20           14.00
    4.49           14.34
    6.44           14.34
    -1.85           14.35
    9.54           11.76
    0.44           13.85
    17.55           14.34
    -31.37           14.30
    10.42            9.33
    -25.96           14.34
    5.77           13.82
    26

    3.75           14.39
    18.90           14.39
    -35.99           14.39
    -5.51           14.21
    23.41           13.65
    22.84           13.51
    -11.56           14.21
    11.44           14.17
    -1.49           13.61
    12.62           14.32
    60.49           14.35
    18.75           14.35
    -14.12           14.05
    -2.21           14.21
    33.50           14.33
    -5.43           14.37
    52.42           14.27
    7.79           10.49
    -38.97           14.03
    14.42           14.33
    19.32           14.14
    -20.28           13.20
    38.32           14.18
    28.00           14.36
    3.97           14.39
    7.62           14.04
    13.12           14.35
    -70.48           14.29
    42.77           14.34
    18.53           14.32
    -18.17           14.28
    94.55           14.19
    12.27           14.34
    -15.63           14.39
    35.45           14.38
    -27.61           14.12
    -7.09           14.35
    -26.67           12.69
    -2

    -24.92           14.35
    -5.11           14.25
    34.04           14.29
    11.10           14.30
    146.15           14.37
    7.71           14.37
    2.86           14.30
    3.89           12.12
    0.64            8.96
    4.36           14.19
    -28.45           14.26
    -2.30           13.24
    -6.84           14.23
    6.19           14.11
    7.48           13.97
    -37.76           14.39
    3.23           14.25
    14.93           14.19
    155.02           14.24
    2.67           14.17
    -1.98           14.34
    13.67           12.15
    -24.31           14.24
    7.82           14.17
    -76.00           14.38
    31.70           14.34
    -13.02           14.03
    -2.37           14.27
    8.29           13.77
    -31.79           14.30
    10.83           13.68
    51.76           14.35
    2.85           14.38
    -11.01           14.38
    -30.58           14.20
    5.26           14.08
    2.26           13.92
    -13.77           14.35
    6.19      

    -26.81           14.40
    2.45           13.56
    11.78            8.17
    5.70           14.35
    -6.09           13.67
    124.85           14.27
    5.48           14.11
    -33.55           14.11
    -10.46           14.32
    72.15           12.68
    0.55           14.29
    14.71           14.39
    0.83           14.37
    -11.17           14.28
    -22.81           14.35
    -31.71           14.36
    27.08           14.42
    40.89           14.34
    0.25           14.36
    40.85           14.03
    -7.22            9.31
    -29.44           14.25
    17.25           14.27
    -6.45           14.19
    -32.84           14.27
    2.78           10.60
    -27.41           14.36
    -15.00           14.36
    -14.23           14.39
    -16.48           13.31
    -9.79           14.30
    -1.34           14.32
    -4.58           14.36
    19.09           14.24
    3.94           14.31
    13.11           14.35
    145.04           13.82
    83.56           14.21
    20